## Imports

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import scipy.io as sio
from characterDefinitions import getHandwritingCharacterDefinitions
from torchvision.models import resnet50
import numpy as np

## Load the Dataset

In [5]:
dataDirs = ['t5.2019.05.08','t5.2019.11.25','t5.2019.12.09','t5.2019.12.11','t5.2019.12.18',
            't5.2019.12.20','t5.2020.01.06','t5.2020.01.08','t5.2020.01.13','t5.2020.01.15']
charDef = getHandwritingCharacterDefinitions()

In [10]:
all_tensors = []
all_labels = []
for directory in dataDirs:
    
    mat = f'./Datasets/{directory}/singleLetters.mat'
    data = sio.loadmat(mat)
    ctr = 0
    for letter in charDef['charList']:
        t = torch.Tensor(data[f'neuralActivityCube_{letter}'])
        qty = t.shape[0]
        labels = torch.Tensor([ctr]*qty)
        ctr += 1
#         if t.shape[0] == 27:
        all_tensors.append(t)
        all_labels.append(labels)

tensor_data = torch.cat(all_tensors, dim=0)
tensor_data = np.repeat(tensor_data[..., np.newaxis], 3, -1).transpose(-1,-2).transpose(-2,-3)

# tensor_data = tensor_data.transpose(-1,0).transpose(-1,-2)
tensor_labels = torch.cat(all_labels).long()


In [11]:
tensor_data=rgb_data
tensor_data.shape

torch.Size([3627, 3, 201, 192])

In [12]:
from torch.utils.data import random_split


dataset = TensorDataset(tensor_data, tensor_labels)
train_data, test_data = random_split(dataset, [3000, 627])
batch_size = 32
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [13]:
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        # Define the layers of your CNN
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(16 * 201 * 192, 31)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x



In [14]:
class MyCNN(nn.Module):
    def __init__(self, num_classes):
        super(MyCNN, self).__init__()
        self.resnet = resnet50(pretrained=True)
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        x = self.resnet(x)
        return x

In [30]:
# Step 4: Model Compilation
model = MyCNN(31)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [16]:
num_epochs = 100
batch_size = 32

In [31]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)


cuda:0


MyCNN(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0)

In [35]:
for epoch in range(num_epochs):
    model.train()
    print(f'epoch {epoch}')
    for batch in train_dataloader:
        inputs, labels = batch
        inputs = inputs.to(device)
        labels= labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs).to(device)  # Add a channel dimension to the input
        loss = criterion(outputs, labels).to(device)
        loss.backward()
        optimizer.step()
    
    # Step 6: Model Evaluation
    model.eval()
    with torch.no_grad():
        cumulative_accuracy = torch.tensor([]).to(device)
        for batch in test_dataloader:
            inputs, labels = batch
            inputs = inputs.to(device)
            labels = labels.to(device)
            val_outputs = model(inputs).to(device)
            val_loss = criterion(val_outputs, labels).to(device)
            val_predictions = torch.argmax(val_outputs, dim=1).to(device)
            val_accuracy = (val_predictions == labels).float().to(device)
            cumulative_accuracy = torch.cat([cumulative_accuracy,val_accuracy], dim=0).to(device)
    
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss.item():.4f}, Validation Accuracy: {cumulative_accuracy.mean().item():.4f}")

epoch 0
Epoch 1/100, Validation Loss: 7.6264, Validation Accuracy: 0.0750
epoch 1
Epoch 2/100, Validation Loss: 2.6868, Validation Accuracy: 0.3349
epoch 2
Epoch 3/100, Validation Loss: 2.3940, Validation Accuracy: 0.3541
epoch 3
Epoch 4/100, Validation Loss: 1.6621, Validation Accuracy: 0.4769
epoch 4
Epoch 5/100, Validation Loss: 1.2855, Validation Accuracy: 0.5470
epoch 5
Epoch 6/100, Validation Loss: 1.1293, Validation Accuracy: 0.6093
epoch 6
Epoch 7/100, Validation Loss: 0.8178, Validation Accuracy: 0.6746
epoch 7
Epoch 8/100, Validation Loss: 0.9926, Validation Accuracy: 0.5726
epoch 8
Epoch 9/100, Validation Loss: 1.5253, Validation Accuracy: 0.6459
epoch 9
Epoch 10/100, Validation Loss: 0.8127, Validation Accuracy: 0.7018
epoch 10
Epoch 11/100, Validation Loss: 1.7683, Validation Accuracy: 0.6746
epoch 11
Epoch 12/100, Validation Loss: 1.6360, Validation Accuracy: 0.6443
epoch 12
Epoch 13/100, Validation Loss: 0.7579, Validation Accuracy: 0.6651
epoch 13
Epoch 14/100, Validati